In [35]:
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [50]:
url = 'https://movie.douban.com/tag/#/?sort=S&range=2,10&tags=%E7%94%B5%E8%A7%86%E5%89%A7,%E9%A6%99%E6%B8%AF'
ips = ['122.152.196.126','114.215.174.227','119.185.30.75']
User_Agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'
    'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
    'Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11',
    'Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11',
]
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9,zh-TW;q=0.8',
    'Connection': 'keep-alive',
    'Cookie': 'll="118287"; __yadk_uid=fqOQ5SoQ2DmjKnVxMJgV6sGtTtmlD8Ax; _vwo_uuid_v2=D0D74B91BC01D9830AEB25AB1D42BF36|ec3fbc275dc554e9dba7d2dfffcd0741; bid=Z7Vi410BK48; douban-fav-remind=1; __utmc=30149280; as="https://www.douban.com/"; _ga=GA1.2.1212773913.1505495235; _gid=GA1.2.2042924453.1542952460; ps=y; dbcl2="162469115:kK7DwQ4IIGE"; ck=PKFL; push_noty_num=0; push_doumail_num=0; __utmz=30149280.1542952477.6.2.utmcsr=open.weixin.qq.com|utmccn=(referral)|utmcmd=referral|utmcct=/connect/qrconnect; __utmv=30149280.16246; douban-profile-remind=1; __utmc=223695111; __utmz=223695111.1542953035.3.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); _pk_ses.100001.4cf6=*; __utma=30149280.1212773913.1505495235.1542952477.1542957924.7; __utmb=30149280.0.10.1542957924; __utma=223695111.1202762546.1505495235.1542953035.1542957924.4; __utmb=223695111.0.10.1542957924; _pk_id.100001.4cf6=e3aecd84d10e1108.1505495235.4.1542957939.1542953035.',
    'Host': 'movie.douban.com',
    'Referer': 'https://movie.douban.com/subject/26425063/',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'
}

In [40]:
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

WebDriverException: Message: unknown error: failed to change window state to maximized, current state is normal
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [28]:
def one_click():
    driver.find_element_by_class_name('more').click()
    time.sleep(1.5)
while True:
    try:
        one_click()
    except:
        break

In [29]:
driver.find_element_by_class_name('th-list').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","selector":"th-list"}
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [6]:
urls = [k.get_attribute('href') for k in driver.find_elements_by_class_name('title')]

In [44]:
len(urls)

1041

In [53]:
def scrape_one_url(url):
    r = requests.get(url, proxies = {'http': random.choice(ips)}, headers = headers)
    data = BeautifulSoup(r.text, 'html.parser')
    my1 = data.find('h1').text.strip()
    my2 = data.find(id = 'info').text.strip()
    my3 = data.find(typeof = 'v:Rating').text.strip()
    return my1, my2, my3
datasets = []
error_urls = []
for url in urls:
    try:
        datasets.append(scrape_one_url(url))
    except:
        error_urls.append(url)

In [54]:
len(datasets)

1040

In [55]:
titles = []
years = []
for dataset in datasets:
    titles.append(dataset[0].split()[0])
    years.append(dataset[0].split()[-1].strip('()'))
df = pd.DataFrame({'剧名': titles, '年份': years})

In [56]:
headers = ['导演', '编剧', '主演', '类型', '制片国家/地区', '集数', '单集片长']
def add_one_df(string):
    things = []
    for dataset in datasets:
        thing = None
        for i in dataset[1].split('\n'):
            if i.find(string) != -1:
                thing = i.strip(string).strip(': ').strip('分钟')
                break
        things.append(thing)
    return things
for header in headers:
    df[header] = add_one_df(header)

In [57]:
scores = []
numbers = []
for dataset in datasets:
    scores.append(dataset[2].split('\n')[0])
    numbers.append(dataset[2].split('\n')[-1].strip('人评价'))
df['豆瓣评分'] = scores
df['评价人数'] = numbers

In [58]:
df.columns = ['剧名', '年份', '导演', '编剧', '主演', '类型', '制片国家/地区', '集数', '单集片长/分钟', '豆瓣评分', '评价人数/人']

In [59]:
final = df.sort_values(by = '豆瓣评分', ascending = False)

In [60]:
final.to_csv('results_Hong Kong.csv', index = False, encoding = 'utf-8-sig')